In [186]:
import os
import subprocess
import numpy as np
import pandas as pd
from astropy.io import ascii
from astropy.table import Table, vstack
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.io import fits
from photutils import CircularAperture
from photutils import aperture_photometry
from astropy import wcs
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia

In [2]:
diffDir = '/media/mj1e16/PP AV-TV/properDiff/'
medianDir = '/media/mj1e16/PP AV-TV/medians/'
imageDir = '/media/mj1e16/PP AV-TV/keplerCal/'

In [3]:
# diffDir = '/data/mj1e16/kepler/properDiff/'
# medianDir = '/data/mj1e16/kepler/medians/'
# imageDir = '/data/mj1e16/kepler/cal/'

In [4]:
diffImages = os.listdir(diffDir)
diffImages = [x for x in diffImages if '.fits' in x]

medianImages = os.listdir(medianDir)
medianImages = [x for x in medianImages if '.fits' in x]

images = os.listdir(imageDir)
images = [x for x in images if '.fits' in x]

In [5]:
unmatchedObjects = ascii.read('/home/mj1e16/outTables15/firstPassTransients15.csv')

In [6]:
xpos = np.array(unmatchedObjects['X_IMAGE'])
ypos = np.array(unmatchedObjects['Y_IMAGE'])
imname = np.array(unmatchedObjects['NAME'])
index = np.array(unmatchedObjects['index'])
allnames = np.unique(imname)

In [7]:
def load_wcs_from_file(filename,pixCoord,ext=0):
    # Load the FITS hdulist using astropy.io.fits
    hdulist = fits.open(filename)
    print(filename)
    # Parse the WCS keywords in the primary HDU
    print(int(ext))
    w = wcs.WCS(hdulist[int(ext)].header)

    # Print out the "name" of the WCS, as defined in the FITS header
    #print(w.wcs.name)

    # Print out all of the settings that were parsed from the header
    w.wcs.print_contents()

    # Three pixel coordinates of interest.
    # Note we've silently assumed a NAXIS=2 image here.
    # Note also that the pixel coordinates are pairs of [X, Y], 
    # and since WCS built from FITS header automatically has
    # origin set to 1, [0, 0] is actually not inside the image.
    pixcrd = np.array(pixCoord, dtype=np.float_)

    # Convert pixel coordinates to world coordinates
    # The second argument is "origin" -- in this case we're declaring we
    # have 1-based (Fortran-like) coordinates.
    world = w.wcs_pix2world(pixcrd, 1)
    #print(world)
    return(world)

In [63]:
def makeDS9RegFile(sexTabList,fileNameBase,tabType,colour='lightgreen',radius='5'):
    for tables in range(len(sexTabList)):
        if tabType == 'dao':
            xcoords = sexTabList[tables]['xcentroid']
            ycoords = sexTabList[tables]['ycentroid']
        elif tabType == 'sexMedian':
            ra = sexTabList[tables]['RA']
            dec = sexTabList[tables]['DEC']
        elif tabType == 'sex':
            xcoords = sexTabList[tables]['X_IMAGE']
            ycoords = sexTabList[tables]['Y_IMAGE']
        elif tabType == 'iraf':
            xcoords = sexTabList[tables]['X_POS']
            ycoords = sexTabList[tables]['Y_POS']        
        else:
            print('tab type error')
            break
        bigString = 'image\n'
        for x in range(len(xcoords)):
            bigString += 'circle({},{},'.format(xcoords[x],ycoords[x])+radius+') # color={}\n'.format(colour)
        fileName = fileNameBase + '.reg'
        with open(fileName,'w') as f:
            f.write(bigString)

In [60]:
def appendToDS9RegFile(sexTabList,fileNameBase,tabType,colour='lightgreen',radius='5'):
    for tables in range(len(sexTabList)):
        if tabType == 'dao':
            xcoords = sexTabList[tables]['xcentroid']
            ycoords = sexTabList[tables]['ycentroid']
        elif tabType == 'sexMedian':
            ra = sexTabList[tables]['RA']
            dec = sexTabList[tables]['DEC']
        elif tabType == 'sex':
            xcoords = sexTabList[tables]['X_IMAGE']
            ycoords = sexTabList[tables]['Y_IMAGE']
        elif tabType == 'iraf':
            xcoords = sexTabList[tables]['X_POS']
            ycoords = sexTabList[tables]['Y_POS']        
        else:
            print('tab type error')
            break
        bigString = ''
        for x in range(len(xcoords)):
            bigString += 'circle({},{},'.format(xcoords[x],ycoords[x])+radius+') # color={}\n'.format(colour)
        fileName = fileNameBase + '.reg'
        with open(fileName,'a') as f:
            f.write(bigString)

In [9]:
def findName(image,diffDir=diffDir,medianDir=medianDir):
    hdu = fits.open(diffDir+image)
    head = hdu[0].header['DIFFCMD']
    locator1 = '/data/mj1e16/kepler/cal/'
    locator2 = ']'
    loc1 = head.find(locator1) + len(locator1)
    loc2 = head.find(locator2) + 1
    imname = head[loc1:loc2]
    
    channellocator1 = 'channel' 
    channellocator2 = '_ccd'
    ccdlocator1 = 'ccd'
    ccdlocator2 = '.fits'
    loc1 = image.find(channellocator1) + len(channellocator1)
    loc2 = image.find(channellocator2)
    channel = image[loc1:loc2]
    loc1 = image.find(ccdlocator1) + len(ccdlocator1)
    loc2 = image.find(ccdlocator2)
    ccd = image[loc1:loc2]
    
    medianName = 'meidian_{}_{}.fits'.format(channel,ccd)
    results = [imname,medianName]
    return results

In [197]:
def findMatchesNoCondition(image,diffname,tab,match,df,bigdf,max_sep = 2.5 * u.arcsec,magsurplus=-3):
    
    
    number1 = match['NUMBER'] # original image
    ra1 = match['RA']
    dec1 = match['DEC']

    number2 = tab['index'] # difference image
    ra2 = tab['RA']
    dec2 = tab['DEC']    
    
    makeDS9RegFile([tab],'/home/mj1e16/ds9RegFiles/foundinBoth','sex',colour='red')

    cat1 = SkyCoord(ra=ra1*u.degree, dec=dec1*u.degree)
    cat2 = SkyCoord(ra=ra2*u.degree, dec=dec2*u.degree)
    
    idx, d2d, d3d = cat2.match_to_catalog_sky(cat1)
    sep_constraint = d2d < max_sep
    objectsFound = number2[sep_constraint]
    matchdf = df[df['ind'].isin(objectsFound)]
    matchdf.sort_values(['Y_IMAGE'])
    ognumber = matchdf['ind'].tolist()
    ogflux = matchdf['FLUX_MAX'].tolist()
    matchTab = Table.from_pandas(matchdf)

    appendToDS9RegFile([matchTab],'/home/mj1e16/ds9RegFiles/foundinBoth','sex',colour='blue',radius='10')
    
    idx, d2d, d3d = cat1.match_to_catalog_sky(cat2)
    sep_constraint = d2d < max_sep
    #istrue = [x for x in sep_constraint if x == True]
    objectsFound = number1[sep_constraint]
    origdf = bigdf[bigdf['NUMBER'].isin(objectsFound)]
    origdf.sort_values(['Y_IMAGE'])
    origflux = origdf['FLUX_MAX'].tolist()
    origMag = origdf['MAG_BEST'].tolist()
    
    fluxdiff = [(x-y) for x,y in zip(origflux,ogflux)]
    magdiff = [-2.5*np.log10(x/y) for x,y in zip(origflux,fluxdiff)]
    #print(magdiff)
    goodNums = [x for x,y in zip(ognumber,magdiff) if y < magsurplus] # fine for now, in future change to not rely on the 
    magdiffColumn = [x for x in magdiff if x < magsurplus]            # Y_iMAGE position of the objects
    magorigColumn = [x for x,y in zip(origMag,magdiff) if y < magsurplus]
    finaldf = matchdf[matchdf['ind'].isin(goodNums)]
    finaldf['magDiff'] = magdiffColumn
    finaldf['MAG_BEST_ORIG'] = magorigColumn
    if len(finaldf) != 0:
        #print(finaldf)
        finalTab = Table.from_pandas(finaldf)
        appendToDS9RegFile([finalTab],'/home/mj1e16/ds9RegFiles/foundinBoth','sex',colour='green',radius='15')
    return finaldf

In [182]:
def showImage(images,regfile):
    commands = ['ds9']
    for x in range(len(images)):
        commands.extend([images[x],'-regions',regfile,'-scale','mode','zscale'])
    commands.extend(['-frame','lock','image'])
    #print(commands)
    subprocess.call(commands)

In [11]:
image2diff = {}
image2median = {}
for x in diffImages:
    results = findName(x)
    image2diff[results[0]] = x
    image2median[results[0]] = results[1]
    
diff = [image2diff[x] for x in imname]
median = [image2median[x] for x in imname]

unmatchedObjects['diffImage'] = diff
unmatchedObjects['medianImage'] = median

In [96]:
uniqueName = np.unique(imname)

In [150]:
bigNames = matches['NAME']
uniqueName = np.unique(bigNames)
magdiff = matches['magDiff']
magdiffsort = sorted(magdiff)
uniqueNameSort = [x for _,x in sorted(zip(magdiffsort,uniqueName))]
print(len(uniqueName))

1079


In [153]:
print(magdiffsort[0])
print(uniqueNameSort[0])

-11.09146127898388
kplr2009114174833_ffi-cal.fits[15]


In [202]:
# goods = [2,10,14,20,32,37,41,44,47,77,84,94,97]
onematch

,ind,NUMBER,MAG_BEST,FLUX_MAX,X_IMAGE,Y_IMAGE,ELONGATION,ELLIPTICITY,RA,DEC,NAME,index,diffImage,medianImage,magDiff,MAG_BEST_ORIG
210655,423122,4,18.2466,342.1980,118.2266,25.9895,1.137,0.120,288.753435,48.463141,kplr2009115002613_ffi-cal.fits[40],423122,diff_imNo37_channel24_ccd40.fits,meidian_24_40.fits,-4.749004,17.3825
210688,423186,262,16.0895,672.9335,541.5507,561.2244,1.284,0.221,288.675277,47.711241,kplr2009115002613_ffi-cal.fits[40],423186,diff_imNo37_channel24_ccd40.fits,meidian_24_40.fits,-4.260063,15.9854


In [203]:
goods = [41,94,97]
for x in goods:
    name = uniqueNameSort[x]
    diffname = image2diff[name]
    df = unmatchedObjects.to_pandas()
    mindf = df[df['NAME'] == name]
    minitab = Table.from_pandas(mindf)
    try:
        matchtab = ascii.read('/data/mj1e16/kepler/sexKeplerCal/'+name+'.csv')
    except:
        matchtab = ascii.read('/media/mj1e16/PP AV-TV/sexKeplerCal/sexKeplerCal/'+name+'.csv')
    #mediantab = ascii.read('/data/mj1e16/kepler/sexKeplerMedian/'+image2median[name]+'.csv')
    bigdf = matchtab.to_pandas()
    #meddf = mediantab.to_pandas()
    onematch = findMatchesNoCondition(imageDir+name,diffDir+diffname,minitab,matchtab,mindf,bigdf,max_sep=5*u.arcsec)
    showImage([imageDir+name,diffDir+image2diff[name],medianDir+image2median[name]],'/home/mj1e16/ds9RegFiles/foundinBoth.reg')
    print(x)

/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel/__main__.py:38: RuntimeWarning: invalid value encountered in log10
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel/__main__.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel/__main__.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


41


IOError: [Errno 2] No such file or directory: '/media/mj1e16/PP AV-TV/sexKeplerCal/sexKeplerCal/kplr2009115131122_ffi-cal.fits[50].csv'

In [ ]:
coord = SkyCoord(ra=280, dec=-60, unit=(u.degree, u.degree), frame='icrs')
width = u.Quantity(0.1, u.deg)
height = u.Quantity(0.1, u.deg)
r = Gaia.query_object_async(coordinate=coord, width=width, height=height)
r.pprint()

In [180]:
onematch = findMatchesNoCondition(imageDir+name,diffDir+diffname,minitab,matchtab,mindf,bigdf,max_sep=5*u.arcsec)
showImage([imageDir+name,diffDir+image2diff[name],medianDir+image2median[name]],'/home/mj1e16/ds9RegFiles/foundinBoth.reg')

/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel/__main__.py:37: RuntimeWarning: invalid value encountered in log10
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel/__main__.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


['ds9', '/media/mj1e16/PP AV-TV/keplerCal/kplr2009114174833_ffi-cal.fits[2]', '-regions', '/home/mj1e16/ds9RegFiles/foundinBoth.reg', '-scale', 'mode', 'zscale', '/media/mj1e16/PP AV-TV/properDiff/diff_imNo11_channel30_ccd2.fits', '-regions', '/home/mj1e16/ds9RegFiles/foundinBoth.reg', '-scale', 'mode', 'zscale', '/media/mj1e16/PP AV-TV/medians/meidian_30_2.fits', '-regions', '/home/mj1e16/ds9RegFiles/foundinBoth.reg', '-scale', 'mode', 'zscale', '-frame', 'lock', 'image']


In [198]:
matches = []
for x in range(len(uniqueName)):
    name = uniqueName[x]
    print(name,x)
    diffname = image2diff[name]
    df = unmatchedObjects.to_pandas()
    mindf = df[df['NAME'] == name]
    minitab = Table.from_pandas(mindf)
    try:
        matchtab = ascii.read('/data/mj1e16/kepler/sexKeplerCal/'+name+'.csv')
    except:
        matchtab = ascii.read('/media/mj1e16/PP AV-TV/sexKeplerCal/sexKeplerCal/'+name+'.csv')
    #mediantab = ascii.read('/data/mj1e16/kepler/sexKeplerMedian/'+image2median[name]+'.csv')
    bigdf = matchtab.to_pandas()
    #meddf = mediantab.to_pandas()

    onematch = findMatchesNoCondition(imageDir+name,diffDir+diffname,minitab,matchtab,mindf,bigdf,max_sep=5*u.arcsec)
    onematchTab = Table.from_pandas(onematch)
    if len(onematch) != 0:
        if len(matches) == 0:
            onematchTab = Table.from_pandas(onematch)
            matches = onematchTab

        else: 
            #onematchTab = Table.from_pandas(onematch)
            #print(onematchTab)
            for x in range(len(onematchTab)):
                matches.add_row(onematchTab[x])
                #print(onematchTab)
                #matches  = vstack(matches,onematchTab)
                #print(matches)
    

('kplr2009114174833_ffi-cal.fits[15]', 0)


/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel/__main__.py:38: RuntimeWarning: invalid value encountered in log10
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel/__main__.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel/__main__.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


('kplr2009114174833_ffi-cal.fits[16]', 1)
('kplr2009114174833_ffi-cal.fits[17]', 2)
('kplr2009114174833_ffi-cal.fits[1]', 3)
('kplr2009114174833_ffi-cal.fits[21]', 4)
('kplr2009114174833_ffi-cal.fits[23]', 5)
('kplr2009114174833_ffi-cal.fits[28]', 6)
('kplr2009114174833_ffi-cal.fits[2]', 7)
('kplr2009114174833_ffi-cal.fits[33]', 8)
('kplr2009114174833_ffi-cal.fits[34]', 9)
('kplr2009114174833_ffi-cal.fits[35]', 10)
('kplr2009114174833_ffi-cal.fits[36]', 11)
('kplr2009114174833_ffi-cal.fits[38]', 12)
('kplr2009114174833_ffi-cal.fits[53]', 13)
('kplr2009114174833_ffi-cal.fits[54]', 14)
('kplr2009114174833_ffi-cal.fits[55]', 15)
('kplr2009114174833_ffi-cal.fits[56]', 16)
('kplr2009114174833_ffi-cal.fits[57]', 17)
('kplr2009114174833_ffi-cal.fits[58]', 18)
('kplr2009114174833_ffi-cal.fits[59]', 19)
('kplr2009114174833_ffi-cal.fits[5]', 20)
('kplr2009114174833_ffi-cal.fits[62]', 21)
('kplr2009114174833_ffi-cal.fits[67]', 22)
('kplr2009114174833_ffi-cal.fits[76]', 23)
('kplr2009114174833_ffi

In [199]:
ascii.write(matches,'/home/mj1e16/outTables15/matchplus3Magdiff.csv',format='csv',fast_writer=False)

In [102]:
showImage([imageDir+name,diffDir+image2diff[name],medianDir+image2median[name]],'/home/mj1e16/ds9RegFiles/foundinBoth.reg')

['ds9', '/media/mj1e16/PP AV-TV/keplerCal/kplr2009114174833_ffi-cal.fits[10]', '-regions', '/home/mj1e16/ds9RegFiles/foundinBoth.reg', '-scale', 'mode', 'zscale', '/media/mj1e16/PP AV-TV/properDiff/diff_imNo11_channel70_ccd10.fits', '-regions', '/home/mj1e16/ds9RegFiles/foundinBoth.reg', '-scale', 'mode', 'zscale', '/media/mj1e16/PP AV-TV/medians/meidian_70_10.fits', '-regions', '/home/mj1e16/ds9RegFiles/foundinBoth.reg', '-scale', 'mode', 'zscale', '-frame', 'lock', 'image']


In [159]:
matches = findMatchesNoCondition(imageDir+name,diffDir+diffname,minitab,matchtab,mindf,bigdf,max_sep=5*u.arcsec)

[-0.05620325995845364, nan, nan, -0.7271288192655224, -0.014472199334716142, -0.05300994831146413, -1.4571815313378544, nan, -0.02180722264547895, -0.011935830200145239, -0.017960166210759618, -0.14517939577093023, -0.022554623422719782, -0.009808813321782576, -1.6606910087271756]


/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in log10


In [16]:
ascii.write(unmatchedObjects,'/home/mj1e16/outTables15/transientsWnames.csv',format='csv',fast_writer=False)